In [1]:
import tushare as ts
import pandas as pd
from bs4 import BeautifulSoup
from IPython.display import display
import requests
import json

path_to_csv = './financial_results.csv'
base_url = 'https://www.tradingview.com'
markets_url = 'https://www.tradingview.com/markets/stocks-china/sectorandindustry-sector/'
ts.set_token('8ef5ec61cdd848715c57c11d58dd71da1271f76b2420d2bac8aef123')
pro = ts.pro_api()

In [2]:
# get sectors
response = requests.get(markets_url)
soup = BeautifulSoup(response.text, 'html.parser')

sectors_table = soup.find('table', {'class': 'tv-data-table'})
sectors = sectors_table.find_all('a', {'class': 'tv-screener__symbol'})

In [4]:
sectors

[<a class="tv-screener__symbol" href="/markets/stocks-china/sectorandindustry-sector/commercial-services/" target="_blank">Commercial Services</a>,
 <a class="tv-screener__symbol" href="/markets/stocks-china/sectorandindustry-sector/communications/" target="_blank">Communications</a>,
 <a class="tv-screener__symbol" href="/markets/stocks-china/sectorandindustry-sector/consumer-durables/" target="_blank">Consumer Durables</a>,
 <a class="tv-screener__symbol" href="/markets/stocks-china/sectorandindustry-sector/consumer-non-durables/" target="_blank">Consumer Non-Durables</a>,
 <a class="tv-screener__symbol" href="/markets/stocks-china/sectorandindustry-sector/consumer-services/" target="_blank">Consumer Services</a>,
 <a class="tv-screener__symbol" href="/markets/stocks-china/sectorandindustry-sector/distribution-services/" target="_blank">Distribution Services</a>,
 <a class="tv-screener__symbol" href="/markets/stocks-china/sectorandindustry-sector/electronic-technology/" target="_blan

In [5]:
# getting company name using the API

def get_largets_company_name(sector):
    filters = {
       "filter":[
          {
             "left":"sector",
             "operation":"equal",
             "right": sector
          }
       ],
       "options":{
          "lang":"en"
       },
       "columns":[
          "name",
          "volume",
          "type",
          "subtype"
       ],
       "sort":{
          "sortBy":"volume",
          "sortOrder":"desc"
       },
       "range":[0,1]
    }
    resp = requests.post('https://scanner.tradingview.com/china/scan', data=json.dumps(filters))
    json_data = resp.json()
    return json_data['data'][0]['d'][0]

In [6]:
def get_finacial_info(company_name):
    # try SH market
    ts_code = f'{company_name}.SH'
    data_frame = pro.query('fina_indicator', ts_code=ts_code, start_date='20200101', end_date='20200828')
    if len(data_frame):
        return data_frame

    # try SZ market
    ts_code = f'{company_name}.SZ'
    data_frame = pro.query('fina_indicator', ts_code=ts_code, start_date='20200101', end_date='20200828')
    if len(data_frame):
        return data_frame

    return None


In [7]:
# Start to get the info

df_results = []

for sector in sectors:
    sector_name = sector.get_text()
    company_name = get_largets_company_name(sector_name)
    financial_info_df = get_finacial_info(company_name)
    df_results.append(financial_info_df)

final_df = pd.concat(df_results)
display(final_df)

,ts_code,ann_date,end_date,eps,dt_eps,total_revenue_ps,revenue_ps,capital_rese_ps,surplus_rese_ps,undist_profit_ps,...,ocf_yoy,roe_yoy,bps_yoy,assets_yoy,eqt_yoy,tr_yoy,or_yoy,q_sales_yoy,q_op_qoq,equity_yoy
0,300059.SZ,20200812,20200630,0.2244,0.2229,0.4141,0.1572,0.9458,0.0544,0.8343,...,-25.8512,75.3299,-10.2162,42.3884,13.0510,67.0901,87.1731,73.5208,5.5442,18.4563
1,300059.SZ,20200812,20200630,0.2244,0.2229,0.4141,0.1572,0.9458,0.0544,0.8343,...,-25.8512,75.3299,-10.2162,42.3884,13.0510,67.0901,87.1731,73.5208,5.5442,18.4563
2,300059.SZ,20200418,20200331,0.1301,0.1301,0.2515,0.0967,1.3298,0.0653,0.8922,...,-91.4021,78.4839,4.1504,23.1895,9.4289,82.2097,102.3122,102.3122,121.4759,26.8927
0,300310.SZ,20200822,20200630,0.0200,0.02,1.1290,1.1290,2.5425,0.0477,-1.5460,...,-31.8202,-38.4245,-0.4114,-4.5107,-0.4083,-15.3751,-15.3751,-3.4567,309.5008,9.6679
1,300310.SZ,20200425,20200331,0.0050,0.005,0.4980,0.4980,2.5416,0.0477,-1.5617,...,-56.6492,-76.6799,-0.6973,-6.5861,-0.6977,-26.8207,-26.8207,-26.8207,117.3679,8.9880
0,600166.SH,20200822,20200630,0.0274,None,4.1390,4.1390,1.2040,0.3301,-0.1422,...,136.4819,-28.8369,0.6736,-0.2730,0.6718,4.4367,4.4367,50.9933,242.9167,1.0471
1,600166.SH,20200822,20200630,0.0274,None,4.1390,4.1390,1.2040,0.3301,-0.1422,...,136.4819,-28.8369,0.6736,-0.2730,0.6718,4.4367,4.4367,50.9933,242.9167,1.0471
2,600166.SH,20200430,20200331,-0.0450,None,1.4029,1.4029,1.2040,0.3301,-0.2145,...,-157.6537,-476.916,-1.9949,2.5695,-1.9968,-34.7816,-34.7816,-34.7816,-269.1000,-2.1592
0,002239.SZ,20200828,20200630,-0.0133,-0.0133,0.4651,0.4651,0.2864,0.0189,0.3536,...,-35.3866,-285.873,-1.2248,0.0956,-1.2237,-12.1797,-12.1797,-2.3449,58.1627,0.4922
1,002239.SZ,20200828,20200630,-0.0133,-0.0133,0.4651,0.4651,0.2864,0.0189,0.3536,...,-35.3866,-285.873,-1.2248,0.0956,-1.2237,-12.1797,-12.1797,-2.3449,58.1627,0.4922


In [13]:
for sector in sectors:
    sector_name = sector.get_text()
    print(sector_name)

Commercial Services
Communications
Consumer Durables
Consumer Non-Durables
Consumer Services
Distribution Services
Electronic Technology
Energy Minerals
Finance
Health Services
Health Technology
Industrial Services
Miscellaneous
Non-Energy Minerals
Process Industries
Producer Manufacturing
Retail Trade
Technology Services
Transportation
Utilities


In [9]:
final_df

,ts_code,ann_date,end_date,eps,dt_eps,total_revenue_ps,revenue_ps,capital_rese_ps,surplus_rese_ps,undist_profit_ps,...,ocf_yoy,roe_yoy,bps_yoy,assets_yoy,eqt_yoy,tr_yoy,or_yoy,q_sales_yoy,q_op_qoq,equity_yoy
0,300059.SZ,20200812,20200630,0.2244,0.2229,0.4141,0.1572,0.9458,0.0544,0.8343,...,-25.8512,75.3299,-10.2162,42.3884,13.0510,67.0901,87.1731,73.5208,5.5442,18.4563
1,300059.SZ,20200812,20200630,0.2244,0.2229,0.4141,0.1572,0.9458,0.0544,0.8343,...,-25.8512,75.3299,-10.2162,42.3884,13.0510,67.0901,87.1731,73.5208,5.5442,18.4563
2,300059.SZ,20200418,20200331,0.1301,0.1301,0.2515,0.0967,1.3298,0.0653,0.8922,...,-91.4021,78.4839,4.1504,23.1895,9.4289,82.2097,102.3122,102.3122,121.4759,26.8927
0,300310.SZ,20200822,20200630,0.0200,0.02,1.1290,1.1290,2.5425,0.0477,-1.5460,...,-31.8202,-38.4245,-0.4114,-4.5107,-0.4083,-15.3751,-15.3751,-3.4567,309.5008,9.6679
1,300310.SZ,20200425,20200331,0.0050,0.005,0.4980,0.4980,2.5416,0.0477,-1.5617,...,-56.6492,-76.6799,-0.6973,-6.5861,-0.6977,-26.8207,-26.8207,-26.8207,117.3679,8.9880
0,600166.SH,20200822,20200630,0.0274,None,4.1390,4.1390,1.2040,0.3301,-0.1422,...,136.4819,-28.8369,0.6736,-0.2730,0.6718,4.4367,4.4367,50.9933,242.9167,1.0471
1,600166.SH,20200822,20200630,0.0274,None,4.1390,4.1390,1.2040,0.3301,-0.1422,...,136.4819,-28.8369,0.6736,-0.2730,0.6718,4.4367,4.4367,50.9933,242.9167,1.0471
2,600166.SH,20200430,20200331,-0.0450,None,1.4029,1.4029,1.2040,0.3301,-0.2145,...,-157.6537,-476.916,-1.9949,2.5695,-1.9968,-34.7816,-34.7816,-34.7816,-269.1000,-2.1592
0,002239.SZ,20200828,20200630,-0.0133,-0.0133,0.4651,0.4651,0.2864,0.0189,0.3536,...,-35.3866,-285.873,-1.2248,0.0956,-1.2237,-12.1797,-12.1797,-2.3449,58.1627,0.4922
1,002239.SZ,20200828,20200630,-0.0133,-0.0133,0.4651,0.4651,0.2864,0.0189,0.3536,...,-35.3866,-285.873,-1.2248,0.0956,-1.2237,-12.1797,-12.1797,-2.3449,58.1627,0.4922


In [8]:
# final_df.to_csv(path_to_csv, encoding='utf-8', index=False)